In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import listdir


In [17]:
#Load data into variables from excel file
fname = r'C:\Users\William\Desktop\Data\PMN-PT 011 Direct\060616 PMN-PT 011 300 V\PMN-PT 011  300V 380K H2C box 2.csv'
data = np.loadtxt(fname,delimiter=',',skiprows=1, usecols=(2,3))
t = data[:,0]
T = data[:,1]
#dt = np.mean(np.diff(t))


p = np.polyfit((t[0], t[-1]),(T[0], T[1]),1)
T = T - t*p[0] - p[1]
Tsample = p[1] + p[0]*t[np.round(t.size/2)]


C:\Users\William\Miniconda3\lib\site-packages\ipykernel\__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\William\Miniconda3\lib\site-packages\ipykernel\__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [31]:
def extractTnew(fname):
    #Extracting data from file 
    #fname = r'C:\Users\William\Desktop\Data\PMN-PT 011 Direct\060616 PMN-PT 011 300 V\PMN-PT 011  300V 380K H2C box 2.csv'
    data = np.loadtxt(fname,delimiter=',',skiprows=1, usecols=(2,3))
    t = data[:,0]
    T = data[:,1]
    
    
    #Linear correction
    p = np.polyfit((t[0], t[-1]),(T[0], T[1]),1)
    T = T - t*p[0] - p[1]
    Tsample = p[1] + p[0]*t[np.round(t.size/2)]
    
    #Smoothing
    s = 20
    Tsmooth = np.convolve(T,np.ones(s)/s, mode='valid')
    tsmooth = np.convolve(t,np.ones(s)/s, mode='valid')
    #plt.plot(t,T,'r')
    #plt.plot(tsmooth,Tsmooth, 'b')
    #plt.show()
    
    t = tsmooth
    T = Tsmooth
    dt = np.mean(np.diff(t))
    
    
    #Differential more useful for finding jumps
    dT = np.diff(T)    
    plt.scatter(t[:-1],dT)
    plt.show()
    
    #Alternative edge detector:
    #kernel = np.array([-1, -1, -1, 0, 1, 1, 1])
    #dT = np.convolve(T,kernel, mode='valid')
    #kernel2 = np.ones(5)/5
    #dT = np.convolve(dT,kernel2,mode='same')
    #plt.plot(t[:-kernel.size+1],dT)
    #plt.show()
    
    #Finding the EC jumps
    #Number of values included is no*2
    no = 40
    maxima = np.argpartition(dT,-no)[-no:]
    minima = np.argpartition(dT,no-1)[:no]
    extrema = np.hstack((maxima,minima))

    plt.plot(t,T)
    plt.scatter(t[extrema+1],T[extrema+1])
    plt.show()
    
    extrema2 = extrema
    
    threshold = 3
    threshold = np.round(threshold/dt)
    for ex in extrema2:
        extrema = extrema[np.abs(extrema-ex)>threshold]
        extrema = np.append(extrema, ex)
    
    plt.plot(t,T)
    plt.scatter(t[extrema+1],T[extrema+1])
    plt.show()
        
    
    Tarr = np.array([Tsample])
    
    for ind in extrema:
        tightness = 2 # In seconds
        tightness = np.round(tightness/dt)
        lower = ind+1-tightness
        upper = ind+1+tightness
        region = Tsmooth[lower:upper]
        tregion = tsmooth[lower:upper]
        maxmin = np.array([np.max(region), np.min(region)])
        Tarr = np.hstack((Tarr,maxmin))
        maxminarg = np.array([np.argmax(region), np.argmin(region)])
        plt.plot(tsmooth,Tsmooth)
        plt.scatter(tregion[maxminarg],maxmin)
    plt.show()
    
    #_,idx = np.unique(Tarr,return_index=True)
    #Tarr = Tarr[np.sort(idx)]
    deltaTarr = np.array([Tarr[0],
                         Tarr[1] - Tarr[2],
                         Tarr[3] - Tarr[4],
                         Tarr[5] - Tarr[6],
                         Tarr[7] - Tarr[8]])
    return deltaTarr
    

In [32]:
fname = r'C:\Users\William\Desktop\Data\PMN-PT 011 Direct\070616 PMN-PT 011 500 V\PMN-PT 011 500 V 385K H2C box 2.csv'

In [33]:
extractTnew(fname)

C:\Users\William\Miniconda3\lib\site-packages\ipykernel\__main__.py:12: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\William\Miniconda3\lib\site-packages\matplotlib\collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
C:\Users\William\Miniconda3\lib\site-packages\ipykernel\__main__.py:12: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\William\Miniconda3\lib\site-packages\matplotlib\collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
C:\Users\William\Miniconda3\lib\site-packages\ipykernel\__main__.py:71: DeprecationWarning: using a non-integer number instead of an integer will result in an e

array([ 89.53118916,   0.13959588,   0.13886462,   0.1359874 ,   0.23621306])

C:\Users\William\Miniconda3\lib\site-packages\ipykernel\__main__.py:71: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\William\Miniconda3\lib\site-packages\ipykernel\__main__.py:72: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


array([ 89.53118916,   0.13959588,   0.13886462,   0.1359874 ,   0.23621306])